In [1]:
project_id = 'movie-genre-analysis-le-wagon'

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [11]:
from google.colab import drive
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

client = bigquery.Client(project=project_id)

df_awards = client.query('''
  SELECT *
  FROM `movie-genre-analysis-le-wagon.Secondary_Data.academy_awards_raw`
  WHERE year_film >= 1974 AND
  NOT film = '' AND
  winner = True
''').to_dataframe()

df_awards

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1974,1975,47,ACTOR,Art Carney,Harry and Tonto,True
1,1974,1975,47,ACTOR IN A SUPPORTING ROLE,Robert De Niro,The Godfather Part II,True
2,1974,1975,47,ACTRESS,Ellen Burstyn,Alice Doesn't Live Here Anymore,True
3,1974,1975,47,ACTRESS IN A SUPPORTING ROLE,Ingrid Bergman,Murder on the Orient Express,True
4,1974,1975,47,ART DIRECTION,"Art Direction: Dean Tavoularis, Angelo Graham...",The Godfather Part II,True
...,...,...,...,...,...,...,...
1170,2023,2024,96,SHORT FILM (Live Action),Wes Anderson and Steven Rales,The Wonderful Story of Henry Sugar,True
1171,2023,2024,96,SOUND,Tarn Willers and Johnnie Burn,The Zone of Interest,True
1172,2023,2024,96,VISUAL EFFECTS,"Takashi Yamazaki, Kiyoko Shibuya, Masaki Takah...",Godzilla Minus One,True
1173,2023,2024,96,WRITING (Adapted Screenplay),Written for the screen by Cord Jefferson,American Fiction,True


In [12]:
df_awards = df_awards.drop(columns=['year_film', 'ceremony', 'winner'])
df_awards = df_awards.rename(columns={
    'film': 'movie_title',
    'category': 'award_category'
})

df_awards

,year_ceremony,award_category,name,movie_title
0,1975,ACTOR,Art Carney,Harry and Tonto
1,1975,ACTOR IN A SUPPORTING ROLE,Robert De Niro,The Godfather Part II
2,1975,ACTRESS,Ellen Burstyn,Alice Doesn't Live Here Anymore
3,1975,ACTRESS IN A SUPPORTING ROLE,Ingrid Bergman,Murder on the Orient Express
4,1975,ART DIRECTION,"Art Direction: Dean Tavoularis, Angelo Graham...",The Godfather Part II
...,...,...,...,...
1170,2024,SHORT FILM (Live Action),Wes Anderson and Steven Rales,The Wonderful Story of Henry Sugar
1171,2024,SOUND,Tarn Willers and Johnnie Burn,The Zone of Interest
1172,2024,VISUAL EFFECTS,"Takashi Yamazaki, Kiyoko Shibuya, Masaki Takah...",Godzilla Minus One
1173,2024,WRITING (Adapted Screenplay),Written for the screen by Cord Jefferson,American Fiction


In [13]:
df_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1175 entries, 0 to 1174
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_ceremony   1175 non-null   Int64 
 1   award_category  1175 non-null   object
 2   name            1175 non-null   object
 3   movie_title     1175 non-null   object
dtypes: Int64(1), object(3)
memory usage: 38.0+ KB


In [14]:
null_values = df_awards.isnull().sum()
print("Null Values per Column:\n", null_values)

Null Values per Column:
 year_ceremony     0
award_category    0
name              0
movie_title       0
dtype: int64


In [16]:
import pandas_gbq

dataset_name = 'Secondary_Data'
table_name = 'academy_awards_cleaned'

pandas_gbq.to_gbq(df_awards, f'{dataset_name}.{table_name}', project_id=project_id)

100%|██████████| 1/1 [00:00<00:00, 573.70it/s]
